In [6]:
import numpy as np
import networkx as nx
from rdkit import Chem
import torch
from torch_geometric.data import Data

# Function to convert SMILES to a NetworkX graph
def smiles_to_graph(smiles):
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return None
    graph = nx.Graph()
    for atom in mol.GetAtoms():
        graph.add_node(atom.GetIdx(), symbol=atom.GetSymbol(), atomic_num=atom.GetAtomicNum())
    for bond in mol.GetBonds():
        graph.add_edge(bond.GetBeginAtomIdx(), bond.GetEndAtomIdx(), bond_type=bond.GetBondType())
        # Add self-loop for each atom
        graph.add_edge(bond.GetBeginAtomIdx(), bond.GetBeginAtomIdx(), bond_type='self')
        graph.add_edge(bond.GetEndAtomIdx(), bond.GetEndAtomIdx(), bond_type='self')
    return graph

# Function to create an adjacency matrix from a NetworkX graph (sparse representation)
def create_adjacency_matrix(graph):
    return nx.adjacency_matrix(graph)

# Function to create node features based on the specified dimensions
def create_node_features(mol):
    atom_features = []
    for atom in mol.GetAtoms():
        element = atom.GetSymbol()
        element_one_hot = [0] * 44
        element_list = ['C', 'N', 'O', 'S', 'F', 'Si', 'P', 'Cl', 'Br', 'Mg', 'Na', 'Ca', 'Fe', 'As',
                        'Al', 'I', 'B', 'V', 'K', 'Tl', 'Yb', 'Sb', 'Sn', 'Ag', 'Pd', 'Co', 'Se',
                        'Ti', 'Zn', 'H', 'Li', 'Ge', 'Cu', 'Au', 'Ni', 'Cd', 'In', 'Mn', 'Zr', 'Cr',
                        'Pt', 'Hg', 'Pb', 'X']
        if element in element_list:
            element_one_hot[element_list.index(element)] = 1

        degree = atom.GetDegree()
        degree_one_hot = [0] * 11
        if degree < 11:
            degree_one_hot[degree] = 1

        total_num_h = atom.GetTotalNumHs()
        total_num_h_one_hot = [0] * 11
        if total_num_h < 11:
            total_num_h_one_hot[total_num_h] = 1

        implicit_h = atom.GetNumImplicitHs()
        implicit_h_one_hot = [0] * 11
        if implicit_h < 11:
            implicit_h_one_hot[implicit_h] = 1

        aromatic = [1] if atom.GetIsAromatic() else [0]

        atom_features.append(element_one_hot + degree_one_hot + total_num_h_one_hot + implicit_h_one_hot + aromatic)

    return np.array(atom_features)

# Process data to PyTorch Geometric Data format
def process_data(smiles_list, target_list):
    data_list = []
    for smiles, target in zip(smiles_list, target_list):
        mol = Chem.MolFromSmiles(smiles)
        if mol is None:
            continue
        graph = smiles_to_graph(smiles)
        x = torch.tensor(create_node_features(mol), dtype=torch.float)
        adj_matrix = create_adjacency_matrix(graph)
        edge_index = torch.tensor(adj_matrix.nonzero(), dtype=torch.long)
        y = torch.tensor([target], dtype=torch.float)  # Target value
        data = Data(x=x, edge_index=edge_index, y=y)
        data_list.append(data)
    return data_list

# Example usage with a list of SMILES strings and dummy target values
smiles_list = ['CC(=O)NC1=CC=C(C=C1)O', 'CCO', 'CCC']
target_list = [1.0, 0.5, 0.8]  # Dummy target values, replace with real target values
graph_data_list = process_data(smiles_list, target_list)

# Save the processed data if needed for later use
torch.save(graph_data_list, 'graph_data_list.pt')

# Print out the node features for verification
for i, data in enumerate(graph_data_list):
    print(f"Graph {i + 1} for SMILES {smiles_list[i]}:")
    print(f"Node features: \n{data.x}")
    print(f"Edge indices: \n{data.edge_index}")
    print(f"Target value: \n{data.y}")


Graph 1 for SMILES CC(=O)NC1=CC=C(C=C1)O:
Node features: 
tensor([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
         0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
         0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,

/tmp/ipykernel_10842/2392957376.py:24: FutureWarning: adjacency_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
  return nx.adjacency_matrix(graph)


In [8]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch_geometric.data import DataLoader
from torch_geometric.nn import GCNConv

# GCN based model
class GNNNet(nn.Module):
    def __init__(self, n_output=1, num_features_mol=78, output_dim=128, dropout=0.2):
        super(GNNNet, self).__init__()
        self.n_output = n_output
        self.mol_conv1 = GCNConv(num_features_mol, num_features_mol)
        self.mol_conv2 = GCNConv(num_features_mol, num_features_mol * 2)
        self.mol_conv3 = GCNConv(num_features_mol * 2, num_features_mol * 4)
        self.mol_fc_g1 = nn.Linear(num_features_mol * 4, 1024)
        self.mol_fc_g2 = nn.Linear(1024, output_dim)

        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(dropout)
        self.fc1 = nn.Linear(output_dim, 1024)
        self.fc2 = nn.Linear(1024, 512)
        self.out = nn.Linear(512, self.n_output)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        x = self.mol_conv1(x, edge_index)
        x = self.relu(x)
        x = self.mol_conv2(x, edge_index)
        x = self.relu(x)
        x = self.mol_conv3(x, edge_index)
        x = self.relu(x)
        x = torch_geometric.nn.global_mean_pool(x, data.batch)

        x = self.relu(self.mol_fc_g1(x))
        x = self.dropout(x)
        x = self.mol_fc_g2(x)
        x = self.dropout(x)

        xc = self.relu(self.fc1(x))
        xc = self.dropout(xc)
        xc = self.relu(self.fc2(xc))
        xc = self.dropout(xc)
        out = self.out(xc)
        return out

# Load the processed data
graph_data_list = torch.load('graph_data_list.pt')

# Create a DataLoader for the graph data
batch_size = 32
data_loader = DataLoader(graph_data_list, batch_size=batch_size, shuffle=True)

# Instantiate the model
model = GNNNet()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Define loss function and optimizer
loss_fn = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 15
model.train()
for epoch in range(num_epochs):
    for batch in data_loader:
        batch = batch.to(device)
        optimizer.zero_grad()
        output = model(batch)  # Feed the batch data to the model
        loss = loss_fn(output, batch.y)  # Assuming batch.y contains the target values
        loss.backward()
        optimizer.step()
    print(f'Epoch {epoch + 1}, Loss: {loss.item()}')


Epoch 1, Loss: 0.6951566338539124
Epoch 2, Loss: 0.4903331995010376
Epoch 3, Loss: 0.28862079977989197
Epoch 4, Loss: 0.11772533506155014
Epoch 5, Loss: 0.08908358216285706
Epoch 6, Loss: 0.13373464345932007
Epoch 7, Loss: 0.05999908968806267
Epoch 8, Loss: 0.04295945167541504
Epoch 9, Loss: 0.05073872208595276
Epoch 10, Loss: 0.05724620819091797
Epoch 11, Loss: 0.0748167335987091
Epoch 12, Loss: 0.06121119484305382
Epoch 13, Loss: 0.04314785450696945
Epoch 14, Loss: 0.0441899374127388
Epoch 15, Loss: 0.0435994528234005
